In [1]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.LightGbm"
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;
using System;
using System.IO;
using System.Linq;
using System.Net;

Installed package Microsoft.ML version 1.4.0

Installed package Microsoft.ML.LightGbm version 1.4.0

In [6]:
if (!File.Exists("arrhythmia.data"))
{
    using var client = new WebClient();
    client.DownloadFile("https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data", "arrhythmia.data");
}
Console.WriteLine($"Data file has {File.ReadLines("arrhythmia.data").Count():n0} lines");
File.ReadLines("arrhythmia.data").Take(5)

Data file has 452 lines


index,value
0,"75,0,190,80,91,193,371,174,121,-16,13,64,-2,?,63,0,52,44,0,0,32,0,0,0,0,0,0,0,44,20,36,0,28,0,0,0,0,0,0,52,40,0,0,0,60,0,0,0,0,0,0,52,0,0,0,0,0,0,0,0,0,0,0,0,56,36,0,0,32,0,0,0,0,0,0,48,32,0,0,0,56,0,0,0,0,0,0,80,0,0,0,0,0,0,0,0,0,0,0,0,40,52,0,0,28,0,0,0,0,0,0,0,48,48,0,0,32,0,0,0,0,0,0,0,52,52,0,0,36,0,0,0,0,0,0,0,52,48,0,0,32,0,0,0,0,0,0,0,56,44,0,0,32,0,0,0,0,0,0,-0.2,0.0,6.1,-1.0,0.0,0.0,0.6,2.1,13.6,30.8,0.0,0.0,1.7,-1.0,0.6,0.0,1.3,1.5,3.7,14.5,0.1,-5.2,1.4,0.0,0.0,0.0,0.8,-0.6,-10.7,-15.6,0.4,-3.9,0.0,0.0,0.0,0.0,-0.8,-1.7,-10.1,-22.0,0.0,0.0,5.7,-1.0,0.0,0.0,-0.1,1.2,14.1,22.5,0.0,-2.5,0.8,0.0,0.0,0.0,1.0,0.4,-4.8,-2.7,0.1,-6.0,0.0,0.0,0.0,0.0,-0.8,-0.6,-24.0,-29.7,0.0,0.0,2.0,-6.4,0.0,0.0,0.2,2.9,-12.6,15.2,-0.1,0.0,8.4,-10.0,0.0,0.0,0.6,5.9,-3.9,52.7,-0.3,0.0,15.2,-8.4,0.0,0.0,0.9,5.1,17.7,70.7,-0.4,0.0,13.5,-4.0,0.0,0.0,0.9,3.9,25.5,62.9,-0.3,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8"
1,"56,1,165,64,81,174,401,149,39,25,37,-17,31,?,53,0,48,0,0,0,24,0,0,0,0,0,0,0,64,0,0,0,24,0,0,0,0,0,0,32,24,0,0,0,40,0,0,0,0,0,0,48,0,0,0,0,0,0,0,0,0,0,0,0,44,20,0,0,24,0,0,0,0,0,0,0,60,0,0,0,20,0,0,0,0,0,0,0,24,52,0,0,16,0,0,0,0,0,0,0,32,52,0,0,20,0,0,0,0,0,0,0,44,48,0,0,32,0,0,0,0,0,0,0,48,44,0,0,32,0,0,0,0,0,0,0,48,40,0,0,28,0,0,0,0,0,0,0,48,0,0,0,28,0,0,0,0,0,0,-0.6,0.0,7.2,0.0,0.0,0.0,0.4,1.5,17.2,26.5,0.0,0.0,5.5,0.0,0.0,0.0,0.1,1.7,17.6,29.5,0.3,-1.6,0.9,0.0,0.0,0.0,-0.3,0.4,-1.5,1.3,0.1,-6.4,0.0,0.0,0.0,0.0,-0.3,-1.6,-15.3,-25.5,-0.3,0.0,4.2,-0.9,0.0,0.0,0.4,0.7,8.3,12.3,0.2,0.0,2.2,0.0,0.0,0.0,-0.2,0.8,6.6,11.7,0.4,0.0,1.0,-8.8,0.0,0.0,0.5,-0.6,-21.6,-26.8,0.4,0.0,2.6,-7.9,0.0,0.0,0.8,2.0,-16.4,1.2,0.0,0.0,5.8,-7.7,0.0,0.0,0.9,3.8,-5.7,27.7,-0.2,0.0,9.5,-5.0,0.0,0.0,0.5,2.6,11.8,34.6,-0.4,0.0,11.0,-2.4,0.0,0.0,0.4,2.6,21.6,43.4,-0.5,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6"
2,"54,0,172,95,138,163,386,185,102,96,34,70,66,23,75,0,40,80,0,0,24,0,0,0,0,0,0,20,56,52,0,0,40,0,0,0,0,0,0,28,116,0,0,0,52,0,0,0,0,0,0,52,64,0,0,0,88,0,0,0,0,0,0,0,36,92,0,0,24,0,0,0,0,0,0,0,128,0,0,0,24,0,1,0,0,0,0,0,24,36,76,0,100,0,0,0,0,0,0,0,40,28,60,0,96,0,0,0,0,0,0,0,48,20,56,24,32,0,0,0,0,0,0,0,44,88,0,0,28,0,0,0,0,0,0,0,44,76,0,0,28,0,0,0,0,0,0,0,44,72,0,0,24,0,0,0,0,0,0,1.0,0.0,4.5,-2.8,0.0,0.0,0.3,2.5,-2.2,19.8,0.8,-0.4,6.4,-1.3,0.0,0.0,0.7,2.7,14.2,37.9,-0.2,-0.6,4.4,0.0,0.0,0.0,0.5,0.2,24.7,26.2,-1.0,-5.3,1.8,0.0,0.0,0.0,-0.5,-2.5,-8.0,-28.5,0.5,0.0,1.7,-2.7,0.0,0.0,-0.2,1.0,-9.4,-1.2,0.4,0.0,4.9,0.0,0.0,0.0,0.6,1.4,31.3,42.7,-0.8,0.0,0.7,-3.8,6.5,0.0,0.3,-3.3,18.7,-13.6,-0.9,0.0,2.2,-4.1,7.4,0.0,0.5,-2.4,20.9,-2.6,0.0,0.0,5.8,-4.1,4.0,-0.5,0.4,0.3,20.4,23.3,0.7,0.0,10.0,-5.7,0.0,0.0,0.5,2.2,-3.0,20.7,1.3,0.0,11.1,-3.4,0.0,0.0,0.4,3.4,11.5,48.2,0.9,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10"
3,"55,0,175,94,100,202,380,179,143,28,11,-5,20,?,71,0,72,20,0,0,48,0,0,0,0,0,0,0,64,36,0,0,36,0,0,0,0,0,0,20,52,48,0,0,56,0,0,0,0,0,0,64,32,0,0,0,72,0,0,0,0,0,0,0,60,12,0,0,44,0,0,0,0,0,0,0,60,44,0,0,32,0,0,0,0,0,0,56,0,0,0,0,0,0,0,0,0,0,0,0,40,44,0,0,20,0,0,0,0,0,0,0,52,40,0,0,32,0,0,0,0,0,0,0,56,48,0,0,36,0,0,0,0,0,0,0,60,48,0,0,36,0,0,0,0,0,0,0,64,40,0,0,40,0,0,0,0,0,0,0.9,0.0,7.8,-0.7,0.0,0.0,1.1,1.9,27.3,45.1,0.1,0.0,9.1,-2.6,0.0,0.0,0.4,1.5,24.5,36.8,-0.4,-0.4,1.6,-2.2,0.0,0.0,-1.0,-0.9,-1.5,-9.2,-0.4,-8.2,1.8,0.0,0.0,0.0,-0.7,-1.7,-23.4,-35.6,0.9,0.0,3.2,-0.4,0.0,0.0,0.7,1.2,9.4,18.0,-0.1,0.0,5.1,-2.5,0.0,0.0,0.3,0.6,9.8,12.6,1.6,-6.5,0.0,0.0,0.0,0.0,-0.4,-0.4,-18.2,-22.4,2.1,0.0,1.2,-6.9,0.0,0.0,-0.5,2.9,-12.7,18.0,0.7,0.0,9.0,-7.9,0.0,0.0,0.1,4.1,7.6,51.0,0.4,0.0,15.0,-5.5,0.0,0.0,0.1,3.3,28.8,63.1,0.1,0.0,15.2,-3.7,0.0,0.0,0.6,3.0,36.8,68.0,0.1,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1"
4,"75,0,190,80,88,181,360,177,103,-16,13,61,3,?,?,0,48,40,0,0,28,0,0,0,0,0,0,0,40,24,0,0,24,0,0,0,0,0,0,52,36,0,0,0,60,0,0,0,0,0,0,48,28,0,0,0,56,0,0,0,0,0,0,0,48,36,0,0,28,0,0,0,0,0,0,44,0,0,0,0,0,0,0,0,0,0,0,88,0,0,0,0,0,0,0,0,0,0,0,0,40,52,0,0,28,0,0,0,0,0,0,0,48,48,0,0,32,0,0,0,0,0,0,0,48,52,0,0,32,0,0,0,0,0,0,0,52,44,0,0,28,0,0,0,0,0,0,0,52,48,0

In [3]:
var context = new MLContext();

In [4]:
var columns = Enumerable.Range(0, 278)
    .Select(n => new TextLoader.Column($"{n}", DataKind.Single, n))
    .ToList();
columns.Add(new TextLoader.Column("Label", DataKind.Int32, 279));

In [5]:
var featureColumns = Enumerable.Range(0, 278).Select(n => $"{n}").ToArray();

In [7]:
var textLoader = context.Data.CreateTextLoader(columns.ToArray(), hasHeader: false, separatorChar: ',');

In [8]:
var allData = textLoader.Load("arrhythmia.data");
allData = context.Data.ShuffleRows(allData);

In [9]:
var splitData = context.Data.TrainTestSplit(allData, testFraction: 0.2);
var (trainData, testData) = (splitData.TrainSet, splitData.TestSet);

In [15]:
class ArrhythmiaLabel
{
    public int Label { get; set; }
}

context.Data.CreateEnumerable<ArrhythmiaLabel>(allData, reuseRowObject: false)
    .GroupBy(x => x.Label)
    .Select(x => new { Key = x.Key, Count = x.Count() })
    .OrderBy(x => x.Key)

index,Key,Count
0,1,245
1,2,44
2,3,15
3,4,15
4,5,13
5,6,25
6,7,3
7,8,2
8,9,9
9,10,50


In [16]:
var pipeline = context.Transforms.Conversion
    .MapValueToKey("Label", "Label")
    .Append(context.Transforms.Conversion.MapKeyToValue("LabelValue", "Label"))
    .Append(context.Transforms.Concatenate("Features", featureColumns))
    .Append(context.Transforms.ReplaceMissingValues("Features", replacementMode: MissingValueReplacingEstimator.ReplacementMode.Mean));
    //.Append(context.Transforms.NormalizeLogMeanVariance("FeaturesNorm", "Features"))
    //.Append(context.Transforms.ProjectToPrincipalComponents("Features", "FeaturesNorm", rank: 10, ensureZeroMean: false));

In [17]:
var transformer = pipeline.Fit(trainData);

In [18]:
trainData.Preview().RowView.Take(5)

index,Values
0,"[ 0: 35, 1: 0, 2: 170, 3: 80, 4: 81, 5: 135, 6: 337, 7: 152, 8: 79, 9: 71 ... (270 more) ]"
1,"[ 0: 63, 1: 1, 2: 163, 3: 62, 4: 79, 5: 160, 6: 371, 7: 171, 8: 100, 9: 9 ... (270 more) ]"
2,"[ 0: 47, 1: 0, 2: 175, 3: 94, 4: 101, 5: 175, 6: 336, 7: 229, 8: 118, 9: -52 ... (270 more) ]"
3,"[ 0: 52, 1: 1, 2: 155, 3: 104, 4: 84, 5: 188, 6: 450, 7: 193, 8: 89, 9: 22 ... (270 more) ]"
4,"[ 0: 44, 1: 1, 2: 155, 3: 65, 4: 80, 5: 117, 6: 363, 7: 142, 8: 72, 9: 56 ... (270 more) ]"


In [24]:
var transformedTrainData = transformer.Transform(trainData);
transformedTrainData.GetColumn<VBuffer<float>>("Features")
    .Take(5)
    .Select(x => x.DenseValues())

index,
0,"[ 44, 1, 155, 65, 80, 117, 363, 142, 72, 56 ... (268 more) ]"
1,"[ 44, 1, 155, 65, 80, 117, 363, 142, 72, 56 ... (268 more) ]"
2,"[ 44, 1, 155, 65, 80, 117, 363, 142, 72, 56 ... (268 more) ]"
3,"[ 44, 1, 155, 65, 80, 117, 363, 142, 72, 56 ... (268 more) ]"
4,"[ 44, 1, 155, 65, 80, 117, 363, 142, 72, 56 ... (268 more) ]"


In [23]:
var estimator = context.MulticlassClassification.Trainers.LightGbm(featureColumnName: "Features", learningRate: 0.1);

In [25]:
var cvResults = context.MulticlassClassification.CrossValidate(transformedTrainData, estimator, numberOfFolds: 5);
var cvResult = cvResults
    .OrderByDescending(x => x.Metrics.MicroAccuracy)
    .First();

In [26]:
cvResult.Metrics.ConfusionMatrix.GetFormattedConfusionTable()


Confusion table
          ||========================================================================================================
PREDICTED ||     0 |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 | Recall
TRUTH     ||========================================================================================================
        0 ||    33 |     0 |     1 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 0.9706
        1 ||     0 |     2 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 1.0000
        2 ||     3 |     0 |     5 |     0 |     0 |     0 |     0 |     1 |     0 |     0 |     0 |     0 |     0 | 0.5556
        3 ||     0 |     0 |     0 |     1 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 1.0000
        4 ||     0 |     0 |     1 |     0 |     2 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 0.6667
     

In [27]:
cvResult.Metrics

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,PerClassLogLoss,ConfusionMatrix
0.7280922552599305,0.5099024015247334,0.6633986928104575,0.8412698412698413,0,0,"[ 0.10989366726702027, 0.4024623859193353, 2.176753866782354, 0.044070338502660296, 0.9718922284767023, 1.001466015085473, 0, 0.2784653829309627, 3.8662074840408827, 0 ... (3 more) ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.868421052631579, 0.6666666666666666, 0.625, 1, 1, 1, 0, 0.75, 0, 0 ... (3 more) ], PerClassRecall: [ 0.9705882352941176, 1, 0.5555555555555556, 1, 0.6666666666666666, 0.7777777777777778, 0, 1, 0, 0 ... (3 more) ], Counts: [ [ 33, 0, 1, 0, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 0, 2, 0, 0, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 3, 0, 5, 0, 0, 0, 0, 1, 0, 0 ... (3 more) ], [ 0, 0, 0, 1, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 0, 0, 1, 0, 2, 0, 0, 0, 0, 0 ... (3 more) ], [ 2, 0, 0, 0, 0, 7, 0, 0, 0, 0 ... (3 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 0, 0, 0, 0, 0, 0, 0, 3, 0, 0 ... (3 more) ], [ 0, 0, 1, 0, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (3 more) ] ... (3 more) ], NumberOfClasses: 13 }"


In [36]:
 var transformedTestData = transformer.Transform(testData);
var predictions = cvResult.Model.Transform(transformedTestData);
var metrics = context.MulticlassClassification.Evaluate(predictions);
metrics.ConfusionMatrix.GetFormattedConfusionTable()


Confusion table
          ||========================================================================================================
PREDICTED ||     0 |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 | Recall
TRUTH     ||========================================================================================================
        0 ||    47 |     0 |     2 |     1 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 0.9400
        1 ||     0 |     2 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     1 |     0 |     0 | 0.6667
        2 ||     2 |     0 |     4 |     0 |     0 |     1 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 0.5714
        3 ||     2 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 0.0000
        4 ||     0 |     0 |     0 |     0 |     1 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 1.0000
     

In [37]:
metrics

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,PerClassLogLoss,ConfusionMatrix
0.9999169630700782,0.33473492425402496,0.5237662337662337,0.7471264367816092,0,0,"[ 0.19257263602544283, 0.3441131678030298, 0.8739057416881142, 5.02887654786613, 0.33900150120332617, 1.194012155814341, 0, 0.18678455457153084, 1.7146666045522647, 0.06982035166728562 ... (3 more) ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.7704918032786885, 1, 0.6666666666666666, 0, 0.5, 0.875, 0, 0.75, 0, 1 ... (3 more) ], PerClassRecall: [ 0.94, 0.6666666666666666, 0.5714285714285714, 0, 1, 0.5833333333333334, 0, 1, 0, 1 ... (3 more) ], Counts: [ [ 47, 0, 2, 1, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 0, 2, 0, 0, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 2, 0, 4, 0, 0, 1, 0, 0, 0, 0 ... (3 more) ], [ 2, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 0, 0, 0, 0, 1, 0, 0, 0, 0, 0 ... (3 more) ], [ 5, 0, 0, 0, 0, 7, 0, 0, 0, 0 ... (3 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 0, 0, 0, 0, 0, 0, 0, 3, 0, 0 ... (3 more) ], [ 0, 0, 0, 1, 0, 0, 0, 0, 0, 0 ... (3 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1 ... (3 more) ] ... (3 more) ], NumberOfClasses: 13 }"


In [35]:
class MulticlassCLassificationPrediction
{
    public int LabelValue { get; set; }

    public float[] Score { get; set; }

    public int PredictedLabelValue { get; set; }
}
var sampleData = context.Data.ShuffleRows(testData);
var transformedSampleData = transformer.Transform(sampleData);

var samplePredictions = cvResult.Model.Transform(transformedSampleData);
var mapValues = context.Transforms.Conversion
    .MapKeyToValue("PredictedLabelValue", "PredictedLabel")
    .Append(context.Transforms.Conversion.MapKeyToValue("LabelValue", "Label"))
    .Fit(samplePredictions);
samplePredictions = mapValues.Transform(samplePredictions);
var samplePredictionItems = context.Data.CreateEnumerable<MulticlassCLassificationPrediction>(samplePredictions, reuseRowObject: false);

samplePredictionItems.Take(5)

index,LabelValue,Score,PredictedLabelValue
0,1,"[ 0.9911036, 0.0013564104, 0.00031412093, 0.00036543494, 0.0006740943, 0.00087928015, 0.0008608224, 0.0005343373, 0.001471047, 0.0011698928 ... (3 more) ]",1
1,1,"[ 0.97871804, 0.00070204074, 0.000567113, 0.0006725636, 0.00483075, 0.0007381455, 0.00033956888, 0.0005412787, 0.00033670306, 0.0010853928 ... (3 more) ]",1
2,6,"[ 0.27525988, 0.00041724718, 0.0002677037, 0.00028174487, 0.7124814, 0.00028550072, 0.0002869314, 0.0005466677, 0.00038055086, 0.0010021513 ... (3 more) ]",6
3,1,"[ 0.65554106, 0.0057336553, 0.006362197, 0.005412931, 0.0037949493, 0.15666652, 0.017861439, 0.0038959598, 0.012442654, 0.01254902 ... (3 more) ]",1
4,1,"[ 0.7141701, 0.0037454874, 0.0020437965, 0.0012799131, 0.0009538788, 0.036205694, 0.18477418, 0.0011883166, 0.0025142594, 0.0041469703 ... (3 more) ]",1
